### Testing the extraction parameters

In [1]:
using PyCall
using Statistics , StatsBase
using  DataFrames, Printf

import PyPlot
sns= pyimport("seaborn")

using DelimitedFiles

## directory
rootdir = ENV["GAIA_ROOT"]
wdir    = "$rootdir/products"
votdir  = "$rootdir/products/votable"
plotdir = "$rootdir/products/testfull2"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering


cd(wdir)

┌ Info: Precompiling GaiaClustering [top-level]
└ @ Base loading.jl:1260


In [2]:
mutable struct _meta
    rootdir::String
    wdir::String
    votdir::String
    plotdir::String
    ocdir::String
    votname::String
    
    minQ::Float64
    minstars::Int
    maxstars::Int
    forcedminstars::Int
    mingoodsolution::Int
    niterqminq::Int
    nburnout::Int
    niter::Int
    maxiters::Int

    epsmean::Float64
    epsdisp::Float64
    min_nei::Int
    min_cl::Int
    ncoredisp::Int
    w3dmean::Float64
    w3ddisp::Float64
    wvelmean::Float64
    wveldisp::Float64
    whrdmean::Float64
    whrddisp::Float64

    aperture2d::Float64
    maxaperture2d::Float64 
    aperture3d::Float64    
    maxaperture3d::Float64
    aperturev::Float64   
    maxaperturev::Float64
    nboot::Int
    labels::String
    
    _meta()= new()
end

## return the default parameters
##
function _set_default_params()::_meta
    def = _meta()
    
    ## Directories
    def.rootdir =  ENV["GAIA_ROOT"]
    def.wdir    = "$rootdir/e2e_products"
    def.votdir  = "$wdir/votable"
    def.plotdir = "$wdir/plots-select"
    def.ocdir   = "$wdir/ocfull"
    def.votname = "test.vot"

    ## MCMC
    ##
    def.minQ    = 2.7
    def.minstars = 40
    def.maxstars = 10000
    def.forcedminstars = 30
    def.mingoodsolution = 10        # minimum good solution for Q in check_qminqstar
    def.niterqminq = 500            # check_qminqstar..

    def.nburnout  = 2000            # burn-in iterations
    def.niter     = 15000           # MCMC iterations==number of CHAINS
    def.maxiters  = 500000
    

    ## prior settings
    ##
    def.epsmean   = 2.5             # epsilon mean
    def.epsdisp   = 1.5             # epsilon dispersion
    def.min_nei   = 10              # min_nei mean
    def.min_cl    = 15              # min_cl mean
    def.ncoredisp = 10              # dispersion for min_nei and min_cl
    def.w3dmean   = 6.0
    def.w3ddisp   = 4.0
    def.wvelmean  = 6.0
    def.wveldisp  = 4.0
    def.whrdmean  = 2.0
    def.whrddisp  = 1.5

    ## Metrics find_clusters
    def.aperture2d    = 1.5
    def.maxaperture2d = 15
    def.aperture3d    = 3
    def.maxaperture3d = 20
    def.aperturev     = 3.0
    def.maxaperturev  = 20
    def.nboot         = 50          # bootstrap number

    def.labels        = "Qc"        # method to get labels: Qc|Qn
    
    return(def)
end

_set_default_params (generic function with 1 method)

In [3]:
function _read_params(file)
    par= set_default_params()
    t= readdlm(file,comments=true, comment_char='#')
    
    s= size(t)
    for i in 1:s[1]
        println(t[i,:])
        if t[i,2] == "="
            set_param!(par,t[i,1],t[i,3])
        else
            p= t[i,1]
            println("### parsing parameter file could be wrong near $p ...")
        end
    end
    
    return(par)
end

function _set_param!(def, parstr,value)
    if parstr == "rootdir" def.rootdir= value end
    if parstr == "wdir"    def.wdir= value end
    if parstr == "votdir"  def.votdir= value end
    if parstr == "plotdir" def.plotdir= value end
    if parstr == "ocdir"   def.ocdir= value end
    if parstr == "votname" def.votname= value end

    if parstr == "minQ"            def.minQ= value end
    if parstr == "minstars"        def.minstars= value end
    if parstr == "maxstars"        def.maxstars= value end
    if parstr == "forcedminstars"  def.forcedminstars= value end
    if parstr == "mingoodsolution" def.mingoodsolution= value end
    if parstr == "niterqminq"      def.niterqminq= value end
 
    if parstr == "nburnout" def.nburnout= value end
    if parstr == "niter"    def.niter= value end
    if parstr == "maxiters" def.maxiters= value end
    
    if parstr == "epsmean"   def.epsmean= value end
    if parstr == "epsdisp"   def.epsdisp= value end
    if parstr == "min_nei"   def.min_cl= value end
    if parstr == "min_cl"    def.min_cl= value end    
    if parstr == "ncoredisp" def.ncoredisp= value end
    if parstr == "w3dmean"   def.w3dmean= value end
    if parstr == "w3ddisp"   def.w3ddisp= value end
    if parstr == "wvelmean"  def.wvelmean= value end     
    if parstr == "wveldisp"  def.wveldisp= value end
    if parstr == "whrdmean"  def.whrdmean= value end
    if parstr == "whrddisp"  def.whrddisp= value end

    ## Metrics find_clusters
    if parstr == "aperture2d"     def.aperture2d= value end
    if parstr == "maxaperture2d"  def.maxaperture2d= value end
    if parstr == "aperture3d"     def.aperture3d= value end    
    if parstr == "maxaperture3d"  def.maxaperture3d= value end
    if parstr == "aperturev"      def.aperturev= value end
    if parstr == "maxaperturev"   def.maxaperturev= value end     
    if parstr == "nboot"          def.nboot= value end
    
    if parstr == "labels"  def.labels= value end

end

_set_param! (generic function with 1 method)

In [4]:
function _getdata(filevot)
    voname = filevot

    data       = read_votable(voname)
    df         = filter_data(data)
    dfcart     = add_cartesian(df)
 
    return(dfcart)
end

_getdata (generic function with 1 method)

In [5]:
## Main
function main(file)
    p= read_params(file)
    votname= "FSR 0224-2.0deg.vot"
    p.votdir= "/home/stephane/Science/GAIA/products/votable.2020"
    p.votname= votname
    
    dfcart=  _getdata(p.votdir*"/"*p.votname)
    println(p)
end

main (generic function with 1 method)

In [6]:
file= "test.ext"
main(file)

## All parameters set to default...
Any["minQ", "=", 2.7]
Any["minstars", "=", 40]
Any["forcedminstars", "=", 30]
Any["mingoodsolution", "=", 10]
Any["niterqminq", "=", 500]
Any["nburnout", "=", 2000]
Any["nchain", "=", 150000000]
Any["maxiters", "=", 500000]
Any["epsmean", "=", 2.5]
Any["epsdisp", "=", 1.5]
Any["min_nei", "=", 10]
Any["min_cl", "=", 15]
Any["ncoredisp", "=", 10]
Any["w3dmean", "=", 6.0]
Any["w3ddisp", "=", 4.0]
Any["wvelmean", "=", 6.0]
Any["wveldisp", "=", 4.0]
Any["whrdmean", "=", 2.0]
Any["whrddisp", "=", 1.5]
Any["aperture2d", "=", 1.5]
Any["maxaperture2d", "=", 15.0]
Any["aperture3d", "=", 3.0]
Any["maxaperture3d", "=", 20]
Any["aperturev", "=", 3.0]
Any["maxaperturev", "=", 20.0]
Any["nboot", "=", 50.0]
Any["labels", "=", "Qc"]
## Parameters read from test.ext
## Votable /home/stephane/Science/GAIA/products/votable.2020/FSR 0224-2.0deg.vot read
## Filtering done ...
## Stars selected: 68309
## Cartesian transformation done ...
meta("/home/stephane/Science/GAIA",